# Description
Load the T5 model (large) that I trained in t5-trivia and make predictions. 

# Load Saved Model

In [5]:
%tensorflow_version 2.x
!pip install tensorflow-text
from google.colab import auth
auth.authenticate_user()

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import tensorflow as tf
import tensorflow_text  # Required to run exported model.

saved_model_path = "gs://my_t5_bucket/models/large/export/1683096973"

def load_predict_fn(model_path):
  if tf.executing_eagerly():
    print("Loading SavedModel in eager mode.")
    imported = tf.saved_model.load(model_path, ["serve"])
    return lambda x: imported.signatures['serving_default'](tf.constant(x))['outputs'].numpy()
  else:
    print("Loading SavedModel in tf 1.x graph mode.")
    tf.compat.v1.reset_default_graph()
    sess = tf.compat.v1.Session()
    meta_graph_def = tf.compat.v1.saved_model.load(sess, ["serve"], model_path)
    signature_def = meta_graph_def.signature_def["serving_default"]
    return lambda x: sess.run(
        fetches=signature_def.outputs["outputs"].name, 
        feed_dict={signature_def.inputs["inputs"].name: x}
    )

predict_fn = load_predict_fn(saved_model_path)

<function load_predict_fn.<locals>.<lambda> at 0x7fb349dc1b40>


# Prediction

In [9]:
def answer(question):
  return predict_fn([question])[0].decode('utf-8')

for question in ["trivia question: where is the google headquarters?",
                 "trivia question: what is the most populous country in the world?",
                 "trivia question: who are the 4 members of the beatles?",
                 "trivia question: how many teeth do humans have?",
                 "trivia question: what is a rectangle?",
                 "trivia question: what is the average life expenctancy for humans?"]:
    print(answer(question))

in the mountain view blvd building at 30 rockefeller plaza in santa clara, californi
china
billy preston, john lennon, ringo starr, john lennon
thirty two
a thin ring of red and yellow sand
79.7
